# Call Fan Discovery v2 (All-Inclusive)

Build a short-DTE call buy list from scratch in one notebook:

1. Pull underlyings and options chains
2. Score contracts for raw upside with execution realism
3. Apply risk and concentration constraints
4. Output a broker-ready buy list and exports


In [ ]:
import json
import math
import os
import time
import warnings
from datetime import date, datetime
from pathlib import Path
from typing import Any

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import yfinance as yf
from IPython.display import Markdown, display

warnings.filterwarnings("ignore", category=FutureWarning)


def setup_report_style(renderer: str | None = None) -> None:
    pd.set_option("display.max_columns", 120)
    pd.set_option("display.width", 220)
    pd.set_option("display.max_rows", 300)
    chosen_renderer = renderer or os.getenv("PLOTLY_RENDERER", "notebook_connected")
    pio.renderers.default = chosen_renderer
    report_template = go.layout.Template(
        layout=go.Layout(
            font=dict(family="Times New Roman", size=14, color="#111827"),
            title=dict(font=dict(size=20)),
            paper_bgcolor="white",
            plot_bgcolor="white",
            xaxis=dict(
                showgrid=True,
                gridcolor="#E5E7EB",
                zeroline=False,
                linecolor="#111827",
                mirror=True,
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor="#E5E7EB",
                zeroline=False,
                linecolor="#111827",
                mirror=True,
            ),
            legend=dict(
                orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1
            ),
            margin=dict(l=60, r=30, t=70, b=50),
        )
    )
    pio.templates["report"] = report_template
    pio.templates.default = "report"


def display_table(
    df: pd.DataFrame, caption: str = "", format_dict: dict[str, Any] | None = None
) -> None:
    styler = df.style
    if format_dict:
        styler = styler.format(format_dict, na_rep="--")
    if caption:
        styler = styler.set_caption(caption)
    display(styler)


def safe_float(value: Any, default: float | None = np.nan) -> float | None:
    try:
        if value is None:
            return default
        out = float(value)
        if np.isnan(out):
            return default
        return out
    except Exception:
        return default


def norm_cdf(x: float) -> float:
    return 0.5 * (1.0 + math.erf(x / math.sqrt(2.0)))


def norm_pdf(x: float) -> float:
    return math.exp(-0.5 * x * x) / math.sqrt(2.0 * math.pi)


def rsi(series: pd.Series, window: int = 14) -> float | None:
    if series is None or len(series) < window + 2:
        return None
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window).mean().iloc[-1]
    avg_loss = loss.rolling(window).mean().iloc[-1]
    if avg_loss is None or pd.isna(avg_loss):
        return None
    if avg_loss == 0:
        return 100.0
    rs = avg_gain / avg_loss
    return float(100 - (100 / (1 + rs)))


setup_report_style()


In [8]:
# Configuration
MY_TICKERS = [
    "AAPL",  # Apple
    "MSFT",  # Microsoft
    "NVDA",  # Nvidia
    "TSLA",  # Tesla
    "AMZN",  # Amazon
    "GOOGL",  # Alphabet
    "META",  # Meta
    "NFLX",  # Netflix
    "AMD",  # AMD
    "AVGO",  # Broadcom
    "TSM",  # Taiwan Semi
    "MU",  # Micron
    "CMI",  # Cummins
    "SNPS",  # Synopsys
    "GEV",  # GE Vernova
    "KLAC",  # KLA
    "WDC",  # Western Digital
    "PLTR",  # Palantir
]

MY_BUDGET = 15_000
POSITION_CAP_PCT = 0.75
TICKER_CAP_PCT = 0.75
MAX_CORRELATED_EXPOSURE_PCT = 0.85
CORR_THRESHOLD = 0.75

MIN_DTE = 80  # Extended: target ~90 DTE and longer
MAX_DTE = 350  # Extended: allow ~190 and ~338 DTE
MIN_DELTA = 0.30
MAX_DELTA = 0.55
MIN_POP = 0.08  # Looser: was 0.12
MAX_BE_DIST = 0.15
MAX_IV_HV = 2.00  # Looser: was 1.60
MAX_SPREAD = 0.15  # Looser: was 0.10
MIN_OI = 50  # Looser: was 100
MIN_VOLUME = 20  # Looser: was 50
MAX_THETA_DRAIN = 0.10  # Looser: was 0.08

SLIPPAGE_PCT_OF_SPREAD = 0.35
MAX_CONTRACTS_PER_LINE = 4
MAX_LINES = 14
RATE_LIMIT_SLEEP = 0.20

# Visualization and export settings
ENABLE_CHARTS = True
CHART_MODE = "full"  # options: "minimal", "full"
TOP_N_CANDIDATES = 30
EXPORT_CHARTS = True
CHART_EXPORT_FORMATS = ("html", "png")
CHART_HEIGHT = 520
CHART_WIDTH = 980

OUTPUT_DIR = Path("outputs")
RUN_STAMP = datetime.now().strftime("%Y%m%d_%H%M%S")

# Optional manual strike zone overrides (ticker: (low, high))
MANUAL_ZONES: dict[str, tuple[float, float]] = {}

assert MY_BUDGET > 0, "MY_BUDGET must be positive"
assert len(MY_TICKERS) > 0, "Add at least one ticker"
assert CHART_MODE in {"minimal", "full"}, "CHART_MODE must be 'minimal' or 'full'"
assert set(CHART_EXPORT_FORMATS).issubset(
    {"html", "png"}
), "CHART_EXPORT_FORMATS supports 'html' and/or 'png'"
if isinstance(MY_TICKERS, list):
    FAN_TICKERS = {t: MANUAL_ZONES.get(t) for t in MY_TICKERS}
elif isinstance(MY_TICKERS, dict):
    FAN_TICKERS = MY_TICKERS
else:
    raise TypeError("MY_TICKERS must be a list or dict")

display(
    Markdown(
        f"**Budget:** ${MY_BUDGET:,.0f}  |  **Universe:** {len(FAN_TICKERS)} tickers  |  "
        f"**DTE Focus:** {MIN_DTE}-{MAX_DTE} days (approximate)"
    )
)


**Budget:** $15,000  |  **Universe:** 18 tickers  |  **DTE Focus:** 80-350 days (approximate)

In [9]:
# Data and pricing helpers
def _next_earnings_date(tkr: yf.Ticker) -> date | None:
    try:
        info = tkr.info or {}
        for key in ("earningsTimestamp", "earningsTimestampStart"):
            ts = info.get(key)
            if ts:
                return datetime.fromtimestamp(int(ts)).date()
    except Exception:
        pass
    try:
        cal = tkr.calendar
        if cal is not None and not cal.empty:
            vals = pd.to_datetime(cal.iloc[:, 0], errors="coerce").dropna()
            if not vals.empty:
                return vals.iloc[0].date()
    except Exception:
        pass
    return None


def fetch_underlying_metrics(
    tickers: list[str],
    history_period: str = "1y",
    rate_limit_sleep: float = 0.2,
) -> pd.DataFrame:
    rows: list[dict[str, Any]] = []
    for ticker in tickers:
        try:
            t = yf.Ticker(ticker)
            time.sleep(rate_limit_sleep)
            info = t.info or {}
            hist = t.history(period=history_period)
            if hist is None or hist.empty or "Close" not in hist.columns:
                continue
            close = hist["Close"].dropna()
            if close.empty:
                continue
            spot = float(close.iloc[-1])
            log_ret = np.log(close / close.shift(1)).dropna()
            hv_30 = (
                float(log_ret.iloc[-30:].std() * math.sqrt(252))
                if len(log_ret) >= 30
                else np.nan
            )
            ret_1m = (
                float(close.iloc[-1] / close.iloc[-22] - 1)
                if len(close) >= 22
                else np.nan
            )
            ret_3m = (
                float(close.iloc[-1] / close.iloc[-64] - 1)
                if len(close) >= 64
                else np.nan
            )
            ret_6m = (
                float(close.iloc[-1] / close.iloc[-127] - 1)
                if len(close) >= 127
                else np.nan
            )
            next_earn = _next_earnings_date(t)
            rows.append(
                {
                    "ticker": ticker,
                    "spot": spot,
                    "beta": safe_float(info.get("beta"), 1.0),
                    "hv_30": hv_30,
                    "rsi_14": rsi(close, 14),
                    "ret_1m": ret_1m,
                    "ret_3m": ret_3m,
                    "ret_6m": ret_6m,
                    "target_mean": safe_float(info.get("targetMeanPrice")),
                    "target_high": safe_float(info.get("targetHighPrice")),
                    "next_earnings_date": next_earn.isoformat() if next_earn else None,
                }
            )
        except Exception:
            continue
    return pd.DataFrame(rows)


def build_correlation_matrix(
    tickers: list[str], period: str = "6mo", rate_limit_sleep: float = 0.15
) -> pd.DataFrame:
    series = {}
    for ticker in sorted(set(tickers)):
        try:
            t = yf.Ticker(ticker)
            time.sleep(rate_limit_sleep)
            hist = t.history(period=period)
            if hist is None or hist.empty or "Close" not in hist.columns:
                continue
            close = hist["Close"].dropna()
            if len(close) < 40:
                continue
            series[ticker] = np.log(close / close.shift(1)).dropna()
        except Exception:
            continue
    if len(series) < 2:
        return pd.DataFrame()
    returns = pd.DataFrame(series).dropna()
    if returns.empty or returns.shape[1] < 2:
        return pd.DataFrame()
    return returns.corr()


def bs_call_greeks(
    spot: float, strike: float, dte: int, iv: float, r: float = 0.04
) -> dict[str, float]:
    if spot <= 0 or strike <= 0 or dte <= 0 or iv <= 0:
        return {"delta": np.nan, "gamma": np.nan, "theta": np.nan, "vega": np.nan}
    t = dte / 365.0
    sqrt_t = math.sqrt(t)
    d1 = (math.log(spot / strike) + (r + 0.5 * iv * iv) * t) / (iv * sqrt_t)
    d2 = d1 - iv * sqrt_t
    delta = norm_cdf(d1)
    gamma = norm_pdf(d1) / (spot * iv * sqrt_t)
    theta = (
        -(spot * norm_pdf(d1) * iv) / (2 * sqrt_t)
        - r * strike * math.exp(-r * t) * norm_cdf(d2)
    ) / 365.0
    vega = spot * norm_pdf(d1) * sqrt_t / 100.0
    return {
        "delta": float(delta),
        "gamma": float(gamma),
        "theta": float(theta),
        "vega": float(vega),
    }


def pop_log_normal(
    spot: float, breakeven: float, dte: int, iv: float, r: float = 0.04
) -> float:
    if dte <= 0 or iv <= 0 or spot <= 0 or breakeven <= 0:
        return 0.0
    t = dte / 365.0
    z = (math.log(spot / breakeven) + (r - 0.5 * iv * iv) * t) / (iv * math.sqrt(t))
    return float(norm_cdf(z))


def estimate_entry_fill(
    mid: float, bid: float, ask: float, slippage_pct: float
) -> float:
    if not np.isfinite(mid) or mid <= 0:
        return np.nan
    if np.isfinite(bid) and np.isfinite(ask) and ask > 0 and bid > 0 and ask >= bid:
        edge = (ask - bid) * slippage_pct
        return float(min(ask, max(mid, mid + edge)))
    return float(mid)


def _derive_zone(
    spot: float,
    target_mean: float | None,
    target_high: float | None,
    manual: tuple[float, float] | None,
) -> tuple[float, float, str]:
    if manual is not None:
        return float(manual[0]), float(manual[1]), "manual"
    if target_mean is not None and np.isfinite(target_mean) and target_mean > 0:
        lo = round(spot * 0.97, -1)
        hi = round(float(target_mean), -1)
        if (
            target_high is not None
            and np.isfinite(target_high)
            and target_mean < target_high < target_mean * 1.2
        ):
            hi = round(float(target_high), -1)
        if hi <= lo:
            hi = lo + 20
        return float(lo), float(hi), "analyst"
    lo = round(spot * 0.95, -1)
    hi = round(spot * 1.10, -1)
    if hi <= lo:
        hi = lo + 20
    return float(lo), float(hi), "moneyness"


# Visualization helpers
_PNG_EXPORT_WARNING_SHOWN = False


def build_run_id() -> str:
    return datetime.now().strftime("%Y%m%d_%H%M%S")


def _slugify_chart_name(name: str) -> str:
    cleaned = []
    for ch in str(name).lower():
        cleaned.append(ch if ch.isalnum() else "_")
    return "".join(cleaned).strip("_") or "chart"


def _has_columns(df: pd.DataFrame, required: list[str]) -> bool:
    return all(col in df.columns for col in required)


def _chart_skip(message: str) -> None:
    display(Markdown(f"_Chart skipped: {message}_"))


def apply_standard_layout(fig: go.Figure, title: str) -> go.Figure:
    fig.update_layout(
        template="report",
        title=title,
        height=CHART_HEIGHT,
        width=CHART_WIDTH,
        hovermode="closest",
    )
    fig.update_xaxes(automargin=True)
    fig.update_yaxes(automargin=True)
    return fig


def save_figure(fig: go.Figure, name: str, run_id: str) -> None:
    global _PNG_EXPORT_WARNING_SHOWN
    if not EXPORT_CHARTS:
        return

    out_dir = OUTPUT_DIR / f"charts_{run_id}"
    out_dir.mkdir(parents=True, exist_ok=True)
    chart_slug = _slugify_chart_name(name)

    if "html" in CHART_EXPORT_FORMATS:
        fig.write_html(str(out_dir / f"{chart_slug}.html"), include_plotlyjs="cdn")

    if "png" in CHART_EXPORT_FORMATS:
        try:
            fig.write_image(
                str(out_dir / f"{chart_slug}.png"),
                width=CHART_WIDTH,
                height=CHART_HEIGHT,
                scale=2,
            )
        except Exception as exc:
            if not _PNG_EXPORT_WARNING_SHOWN:
                display(
                    Markdown(
                        f"> PNG export unavailable; continuing with HTML exports only. `{exc}`"
                    )
                )
                _PNG_EXPORT_WARNING_SHOWN = True


def show_and_optionally_save(fig: go.Figure | None, name: str, run_id: str) -> None:
    if fig is None:
        return
    fig.show()
    save_figure(fig, name=name, run_id=run_id)


def plot_candidate_score_rank(pool_df: pd.DataFrame, top_n: int = 30) -> go.Figure | None:
    needed = ["ticker", "expiration", "strike", "upside_score", "upside_multiple"]
    if pool_df.empty or not _has_columns(pool_df, needed):
        _chart_skip("candidate score rank requires populated pool columns")
        return None

    work = pool_df.nlargest(top_n, "upside_score").copy()
    work["label"] = (
        work["ticker"].astype(str)
        + " "
        + work["expiration"].astype(str)
        + " "
        + work["strike"].round(0).astype(int).astype(str)
        + "C"
    )
    work = work.sort_values("upside_score", ascending=True)

    fig = px.bar(
        work,
        x="upside_score",
        y="label",
        orientation="h",
        color="ticker",
        hover_data={
            "upside_multiple": ":.2f",
            "dte": True,
            "delta": ":.2f",
            "pop": ":.2%",
        },
    )
    fig.update_traces(
        text=work["upside_multiple"].map(lambda v: f"{v:.2f}x"),
        textposition="outside",
        cliponaxis=False,
    )
    apply_standard_layout(fig, f"Top {len(work)} Candidate Scores")
    fig.update_xaxes(title="Upside Score")
    fig.update_yaxes(title="Contract")
    return fig


def plot_pop_vs_upside(pool_df: pd.DataFrame, top_n: int = 30) -> go.Figure | None:
    needed = ["ticker", "pop", "upside_multiple", "upside_score", "liq_quality"]
    if pool_df.empty or not _has_columns(pool_df, needed):
        _chart_skip("PoP vs Upside requires pop/upside/liq_quality columns")
        return None

    work = pool_df.nlargest(top_n, "upside_score").copy()
    work["bubble_size"] = (work["liq_quality"].fillna(0.2).clip(0.05, 1.0) * 45).round(2)

    fig = px.scatter(
        work,
        x="pop",
        y="upside_multiple",
        color="ticker",
        size="bubble_size",
        hover_data={
            "upside_score": ":.1f",
            "dte": True,
            "delta": ":.2f",
            "iv_hv": ":.2f",
        },
    )
    apply_standard_layout(fig, "PoP vs Upside Multiple")
    fig.update_xaxes(title="Probability of Profit", tickformat=".0%")
    fig.update_yaxes(title="Upside Multiple")
    return fig


def plot_iv_hv_vs_score(pool_df: pd.DataFrame, top_n: int = 30) -> go.Figure | None:
    needed = ["ticker", "iv_hv", "upside_score", "volume", "oi"]
    if pool_df.empty or not _has_columns(pool_df, needed):
        _chart_skip("IV/HV vs score requires iv_hv/upside_score/liquidity columns")
        return None

    work = pool_df.nlargest(top_n, "upside_score").copy()
    work["size_proxy"] = (
        work["oi"].fillna(0).clip(lower=0) + 2.0 * work["volume"].fillna(0).clip(lower=0)
    )
    work["size_proxy"] = work["size_proxy"].clip(lower=1)

    fig = px.scatter(
        work,
        x="iv_hv",
        y="upside_score",
        color="ticker",
        size="size_proxy",
        hover_data={
            "dte": True,
            "delta": ":.2f",
            "pop": ":.2%",
            "spread_pct": ":.2%",
        },
    )
    apply_standard_layout(fig, "IV/HV Ratio vs Upside Score")
    fig.update_xaxes(title="IV/HV")
    fig.update_yaxes(title="Upside Score")
    return fig


def plot_dte_distribution(pool_df: pd.DataFrame) -> go.Figure | None:
    needed = ["dte", "ticker"]
    if pool_df.empty or not _has_columns(pool_df, needed):
        _chart_skip("DTE distribution requires dte/ticker columns")
        return None

    fig = px.histogram(pool_df, x="dte", color="ticker", marginal="box", nbins=20)
    apply_standard_layout(fig, "Candidate DTE Distribution")
    fig.update_xaxes(title="Days to Expiration")
    fig.update_yaxes(title="Contracts")
    return fig


def plot_ticker_pass_heatmap(overview_df: pd.DataFrame) -> go.Figure | None:
    needed = ["ticker", "contracts_fetched", "contracts_scored"]
    if overview_df.empty or not _has_columns(overview_df, needed):
        _chart_skip("ticker pass heatmap requires overview fetched/passed columns")
        return None

    work = overview_df[needed].copy().sort_values("contracts_scored", ascending=False)
    z = np.column_stack([work["contracts_fetched"], work["contracts_scored"]])

    fig = go.Figure(
        data=go.Heatmap(
            z=z,
            x=["Fetched", "Passed"],
            y=work["ticker"],
            colorscale="Blues",
            text=z.astype(int),
            texttemplate="%{text}",
            hovertemplate="Ticker %{y}<br>%{x}: %{z:.0f}<extra></extra>",
        )
    )
    apply_standard_layout(fig, "Ticker Intake vs Filter Pass-Through")
    fig.update_xaxes(title="Stage")
    fig.update_yaxes(title="Ticker")
    return fig


def plot_liquidity_spread(pool_df: pd.DataFrame, top_n: int = 30) -> go.Figure | None:
    needed = ["ticker", "cost_1ct_exec", "spread_pct", "oi", "upside_score"]
    if pool_df.empty or not _has_columns(pool_df, needed):
        _chart_skip("liquidity/spread chart requires cost, spread, oi columns")
        return None

    work = pool_df.nlargest(top_n, "upside_score").copy()
    work["oi_size"] = work["oi"].fillna(0).clip(lower=1)

    fig = px.scatter(
        work,
        x="cost_1ct_exec",
        y="spread_pct",
        color="ticker",
        size="oi_size",
        hover_data={
            "volume": True,
            "upside_score": ":.1f",
            "dte": True,
        },
    )
    apply_standard_layout(fig, "Execution Cost vs Spread Percent")
    fig.update_xaxes(title="Cost per Contract (execution-adjusted)", tickprefix="$")
    fig.update_yaxes(title="Spread %", tickformat=".1%")
    return fig


def plot_flag_frequency(pool_df: pd.DataFrame) -> go.Figure | None:
    if pool_df.empty or "flags" not in pool_df.columns:
        _chart_skip("flag frequency requires flags column")
        return None

    tokens: list[str] = []
    for raw in pool_df["flags"].fillna("--").astype(str):
        if raw.strip() in {"", "--"}:
            continue
        tokens.extend([p.strip() for p in raw.split(",") if p.strip()])

    if not tokens:
        _chart_skip("no candidate flags available for frequency chart")
        return None

    work = (
        pd.Series(tokens, name="flag")
        .value_counts()
        .rename_axis("flag")
        .reset_index(name="count")
        .sort_values("count", ascending=True)
    )

    fig = px.bar(
        work,
        x="count",
        y="flag",
        orientation="h",
        text="count",
        color="count",
        color_continuous_scale="Teal",
    )
    fig.update_coloraxes(showscale=False)
    apply_standard_layout(fig, "Candidate Flag Frequency")
    fig.update_xaxes(title="Count")
    fig.update_yaxes(title="Flag")
    return fig


def plot_portfolio_weight(buy_df: pd.DataFrame) -> go.Figure | None:
    needed = ["ticker", "portfolio_weight", "line_cost"]
    if buy_df.empty or not _has_columns(buy_df, needed):
        _chart_skip("portfolio weights require ticker/weight/line_cost columns")
        return None

    work = buy_df.sort_values("portfolio_weight", ascending=False).copy()
    fig = px.bar(
        work,
        x="ticker",
        y="portfolio_weight",
        color="ticker",
        text=work["portfolio_weight"].map(lambda v: f"{v:.0%}"),
        hover_data={"line_cost": ":,.0f", "qty": True, "dte": True},
    )
    apply_standard_layout(fig, "Portfolio Weight by Position")
    fig.update_xaxes(title="Ticker")
    fig.update_yaxes(title="Portfolio Weight", tickformat=".0%")
    return fig


def plot_theta_contribution(buy_df: pd.DataFrame) -> go.Figure | None:
    needed = ["ticker", "theta_dollars_day"]
    if buy_df.empty or not _has_columns(buy_df, needed):
        _chart_skip("theta contribution requires ticker/theta_dollars_day columns")
        return None

    work = (
        buy_df.groupby("ticker", as_index=False)["theta_dollars_day"]
        .sum()
        .sort_values("theta_dollars_day", ascending=True)
    )
    fig = px.bar(
        work,
        x="theta_dollars_day",
        y="ticker",
        orientation="h",
        color="theta_dollars_day",
        color_continuous_scale="RdBu",
    )
    fig.update_coloraxes(showscale=False)
    apply_standard_layout(fig, "Daily Theta Burn by Ticker")
    fig.update_xaxes(title="Theta Dollars per Day", tickprefix="$", tickformat=",.0f")
    fig.update_yaxes(title="Ticker")
    return fig


def plot_expiration_ladder(buy_df: pd.DataFrame) -> go.Figure | None:
    needed = ["expiration", "ticker", "line_cost"]
    if buy_df.empty or not _has_columns(buy_df, needed):
        _chart_skip("expiration ladder requires expiration/ticker/line_cost columns")
        return None

    work = (
        buy_df.groupby(["expiration", "ticker"], as_index=False)["line_cost"]
        .sum()
        .sort_values("expiration")
    )
    fig = px.bar(work, x="expiration", y="line_cost", color="ticker", barmode="stack")
    apply_standard_layout(fig, "Expiration Ladder (Premium at Risk)")
    fig.update_xaxes(title="Expiration")
    fig.update_yaxes(title="Line Cost", tickprefix="$", tickformat=",.0f")
    return fig


def plot_correlation_matrix(corr_mx: pd.DataFrame, tickers: list[str]) -> go.Figure | None:
    if corr_mx.empty:
        _chart_skip("correlation matrix unavailable (insufficient return history)")
        return None

    available = [t for t in tickers if t in corr_mx.index and t in corr_mx.columns]
    if len(available) < 2:
        _chart_skip("correlation matrix requires at least two selected tickers")
        return None

    work = corr_mx.loc[available, available]
    fig = go.Figure(
        data=go.Heatmap(
            z=work.values,
            x=list(work.columns),
            y=list(work.index),
            zmin=-1,
            zmax=1,
            colorscale="RdBu",
            reversescale=True,
            colorbar=dict(title="Corr"),
            hovertemplate="%{y} vs %{x}<br>Corr: %{z:.2f}<extra></extra>",
        )
    )
    apply_standard_layout(fig, "Correlation Matrix (Selected Tickers)")
    fig.update_xaxes(title="Ticker")
    fig.update_yaxes(title="Ticker")
    return fig


def plot_entry_stop_tp(buy_df: pd.DataFrame) -> go.Figure | None:
    needed = [
        "ticker",
        "expiration",
        "strike",
        "qty",
        "stop_price",
        "entry_fill",
        "tp_50",
        "tp_100",
    ]
    if buy_df.empty or not _has_columns(buy_df, needed):
        _chart_skip("entry/stop/TP chart requires target and stop columns")
        return None

    work = buy_df.copy().sort_values("entry_fill", ascending=True)
    work["label"] = (
        work["ticker"].astype(str)
        + " "
        + work["expiration"].astype(str)
        + " "
        + work["strike"].round(0).astype(int).astype(str)
        + "C x"
        + work["qty"].astype(int).astype(str)
    )

    fig = go.Figure()

    for _, row in work.iterrows():
        fig.add_trace(
            go.Scatter(
                x=[row["stop_price"], row["tp_100"]],
                y=[row["label"], row["label"]],
                mode="lines",
                line=dict(color="#9CA3AF", width=6),
                showlegend=False,
                hoverinfo="skip",
            )
        )

    fig.add_trace(
        go.Scatter(
            x=work["stop_price"],
            y=work["label"],
            mode="markers",
            name="Stop -50%",
            marker=dict(color="#B91C1C", size=9, symbol="x"),
            hovertemplate="%{y}<br>Stop: $%{x:.2f}<extra></extra>",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=work["entry_fill"],
            y=work["label"],
            mode="markers",
            name="Entry",
            marker=dict(color="#111827", size=9, symbol="circle"),
            hovertemplate="%{y}<br>Entry: $%{x:.2f}<extra></extra>",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=work["tp_50"],
            y=work["label"],
            mode="markers",
            name="TP +50%",
            marker=dict(color="#D97706", size=9, symbol="diamond"),
            hovertemplate="%{y}<br>TP +50%: $%{x:.2f}<extra></extra>",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=work["tp_100"],
            y=work["label"],
            mode="markers",
            name="TP +100%",
            marker=dict(color="#047857", size=10, symbol="star"),
            hovertemplate="%{y}<br>TP +100%: $%{x:.2f}<extra></extra>",
        )
    )

    apply_standard_layout(fig, "Entry, Stop, and Take-Profit Map")
    fig.update_xaxes(title="Option Price")
    fig.update_yaxes(title="Position")
    return fig


In [10]:
# Option chain + short-DTE upside scoring
def fetch_fan_candidates(
    ticker: str,
    spot: float,
    strike_range: tuple[float, float],
    min_dte: int,
    max_dte: int,
    min_open_interest: int,
    min_volume: int,
    max_spread_pct: float,
    max_contracts_per_exp: int = 40,
    rate_limit_sleep: float = 0.2,
) -> pd.DataFrame:
    try:
        t = yf.Ticker(ticker)
        time.sleep(rate_limit_sleep)
        all_exps = t.options
        if not all_exps:
            return pd.DataFrame()
    except Exception:
        return pd.DataFrame()

    lo_strike, hi_strike = strike_range
    today = date.today()
    rows: list[dict[str, Any]] = []

    for exp in all_exps:
        try:
            exp_date = datetime.strptime(exp, "%Y-%m-%d").date()
            dte = (exp_date - today).days
            if dte < min_dte or dte > max_dte:
                continue
            time.sleep(rate_limit_sleep)
            chain = t.option_chain(exp)
            frame = chain.calls
            if frame is None or frame.empty:
                continue

            work = frame.copy()
            work["mid"] = (work["bid"].fillna(0) + work["ask"].fillna(0)) / 2.0
            work.loc[work["mid"] <= 0, "mid"] = work["lastPrice"]
            work["spread"] = work["ask"].fillna(0) - work["bid"].fillna(0)
            work["spread_pct"] = np.where(
                work["mid"] > 0, work["spread"] / work["mid"], np.nan
            )
            work = work[
                (work["strike"] >= lo_strike)
                & (work["strike"] <= hi_strike)
                & (work["impliedVolatility"] > 0)
                & (work["mid"] > 0)
                & (work["openInterest"].fillna(0) >= min_open_interest)
                & (work["volume"].fillna(0) >= min_volume)
                & (work["spread_pct"].fillna(max_spread_pct + 1) <= max_spread_pct)
            ]
            if work.empty:
                continue

            work = work.sort_values(
                ["openInterest", "volume", "mid"], ascending=[False, False, True]
            ).head(max_contracts_per_exp)
            for _, row in work.iterrows():
                rows.append(
                    {
                        "ticker": ticker,
                        "expiration": exp,
                        "dte": int(dte),
                        "contract": row.get("contractSymbol"),
                        "strike": safe_float(row.get("strike"), np.nan),
                        "bid": safe_float(row.get("bid"), np.nan),
                        "ask": safe_float(row.get("ask"), np.nan),
                        "mid": safe_float(row.get("mid"), np.nan),
                        "iv": safe_float(row.get("impliedVolatility"), np.nan),
                        "oi": safe_float(row.get("openInterest"), 0.0),
                        "volume": safe_float(row.get("volume"), 0.0),
                        "spread_pct": safe_float(row.get("spread_pct"), np.nan),
                        "spot": float(spot),
                    }
                )
        except Exception:
            continue

    return pd.DataFrame(rows)


def score_short_upside_candidates(
    chain_df: pd.DataFrame, metrics_row: pd.Series, relaxed: bool = False
) -> pd.DataFrame:
    if chain_df.empty:
        return pd.DataFrame()

    min_delta = MIN_DELTA if not relaxed else max(0.20, MIN_DELTA - 0.10)
    max_delta = MAX_DELTA if not relaxed else min(0.70, MAX_DELTA + 0.15)
    min_pop = MIN_POP if not relaxed else max(0.08, MIN_POP - 0.04)
    max_be_dist = MAX_BE_DIST if not relaxed else MAX_BE_DIST + 0.05
    max_iv_hv = MAX_IV_HV if not relaxed else MAX_IV_HV + 0.30
    max_spread = MAX_SPREAD if not relaxed else MAX_SPREAD * 1.6
    min_oi = MIN_OI if not relaxed else max(30, int(MIN_OI * 0.4))
    min_volume = MIN_VOLUME if not relaxed else max(10, int(MIN_VOLUME * 0.4))

    spot = float(metrics_row["spot"])
    hv = (
        float(metrics_row["hv_30"])
        if pd.notna(metrics_row["hv_30"]) and float(metrics_row["hv_30"]) > 0
        else 0.28
    )
    ret_3m = float(metrics_row["ret_3m"]) if pd.notna(metrics_row["ret_3m"]) else 0.0
    rsi_14 = float(metrics_row["rsi_14"]) if pd.notna(metrics_row["rsi_14"]) else 50.0
    beta = max(
        float(metrics_row["beta"]) if pd.notna(metrics_row["beta"]) else 1.0, 0.30
    )
    next_earn = metrics_row.get("next_earnings_date")
    earn_date = (
        datetime.fromisoformat(next_earn).date() if isinstance(next_earn, str) else None
    )

    scored_rows: list[dict[str, Any]] = []
    for _, lg in chain_df.iterrows():
        dte = int(lg["dte"])
        strike = float(lg["strike"])
        iv = float(lg["iv"])
        oi = float(lg["oi"])
        vol = float(lg["volume"])
        spread_pct = float(lg["spread_pct"]) if pd.notna(lg["spread_pct"]) else np.nan
        entry = estimate_entry_fill(
            float(lg["mid"]), float(lg["bid"]), float(lg["ask"]), SLIPPAGE_PCT_OF_SPREAD
        )
        if not np.isfinite(entry) or entry <= 0:
            continue

        greeks = bs_call_greeks(spot=spot, strike=strike, dte=dte, iv=iv)
        delta = greeks["delta"]
        theta = greeks["theta"]
        gamma = greeks["gamma"]
        vega = greeks["vega"]
        if not np.isfinite(delta):
            continue

        breakeven = strike + entry
        be_pct = breakeven / spot - 1
        pop = pop_log_normal(spot, breakeven, dte, iv)
        iv_hv = iv / hv if hv > 0 else np.nan
        theta_ratio = abs(theta) / entry if entry > 0 and np.isfinite(theta) else np.nan

        expected_move = max(hv * math.sqrt(max(dte, 1) / 365.0), 0.02)
        trend_drift = max(ret_3m, 0.0) * min(dte / 63.0, 1.0)
        bull_move_1 = max(0.01, 0.60 * expected_move + trend_drift)
        bull_move_2 = max(0.02, 1.20 * expected_move + trend_drift)

        pnl_1 = max(spot * (1.0 + bull_move_1) - strike, 0.0) - entry
        pnl_2 = max(spot * (1.0 + bull_move_2) - strike, 0.0) - entry
        upside_multiple = max(pnl_1, pnl_2, 0.0) / entry

        gamma_per_dollar = max(gamma * spot / entry, 0.0) if np.isfinite(gamma) else 0.0
        delta_center = (min_delta + max_delta) / 2.0
        delta_half = max((max_delta - min_delta) / 2.0, 0.01)
        delta_fit = max(1.0 - abs(delta - delta_center) / delta_half, 0.0)

        liq_quality = (
            0.50 * min(oi / 1000.0, 1.0)
            + 0.30 * min(vol / 500.0, 1.0)
            + 0.20
            * max(
                1.0
                - (
                    spread_pct / max_spread
                    if np.isfinite(spread_pct) and max_spread > 0
                    else 1.0
                ),
                0.0,
            )
        )

        iv_penalty = max(iv_hv - 1.10, 0.0) if np.isfinite(iv_hv) else 0.5
        theta_penalty = (
            max(theta_ratio - 0.03, 0.0) if np.isfinite(theta_ratio) else 0.5
        )
        be_penalty = max(be_pct - 0.04, 0.0)

        earnings_risk = False
        if earn_date is not None:
            days_to_earn = (earn_date - date.today()).days
            earnings_risk = 0 <= days_to_earn <= dte

        upside_score = (
            65.0 * upside_multiple
            + 20.0 * gamma_per_dollar
            + 10.0 * delta_fit
            + 8.0 * pop
            + 6.0 * liq_quality
            - 18.0 * iv_penalty
            - 25.0 * theta_penalty
            - 20.0 * be_penalty
            - (8.0 if earnings_risk else 0.0)
        )

        gate_reasons = []
        if delta < min_delta or delta > max_delta:
            gate_reasons.append("delta")
        if pop < min_pop:
            gate_reasons.append("pop")
        if be_pct > max_be_dist:
            gate_reasons.append("be")
        if iv_hv > max_iv_hv:
            gate_reasons.append("iv_hv")
        if oi < min_oi:
            gate_reasons.append("oi")
        if vol < min_volume:
            gate_reasons.append("vol")
        if np.isfinite(spread_pct) and spread_pct > max_spread:
            gate_reasons.append("spread")
        if np.isfinite(theta_ratio) and theta_ratio > MAX_THETA_DRAIN:
            gate_reasons.append("theta")

        if gate_reasons:
            continue

        flags = []
        if earnings_risk:
            flags.append("earnings")
        if rsi_14 > 75:
            flags.append("rsi_hot")
        if ret_3m < -0.10:
            flags.append("downtrend")
        if iv_hv > 1.40:
            flags.append("iv_rich")

        scored_rows.append(
            {
                "ticker": lg["ticker"],
                "contract": lg["contract"],
                "expiration": lg["expiration"],
                "dte": dte,
                "strike": strike,
                "spot": spot,
                "mid": float(lg["mid"]),
                "entry_fill": entry,
                "cost_1ct_exec": entry * 100.0,
                "iv": iv,
                "hv": hv,
                "iv_hv": iv_hv,
                "oi": oi,
                "volume": vol,
                "spread_pct": spread_pct,
                "delta": delta,
                "gamma": gamma,
                "theta": theta,
                "vega": vega,
                "beta": beta,
                "breakeven": breakeven,
                "be_pct": be_pct,
                "pop": pop,
                "expected_move": expected_move,
                "bull_move_1": bull_move_1,
                "bull_move_2": bull_move_2,
                "pnl_bull_1": pnl_1,
                "pnl_bull_2": pnl_2,
                "upside_multiple": upside_multiple,
                "gamma_per_dollar": gamma_per_dollar,
                "theta_ratio": theta_ratio,
                "liq_quality": liq_quality,
                "upside_score": upside_score,
                "earnings_risk": earnings_risk,
                "score_mode": "relaxed" if relaxed else "strict",
                "flags": ", ".join(flags) if flags else "--",
            }
        )

    if not scored_rows:
        return pd.DataFrame()

    out = pd.DataFrame(scored_rows)
    return out.sort_values(
        ["upside_score", "upside_multiple", "liq_quality"], ascending=False
    ).reset_index(drop=True)


In [11]:
# Build candidate pool
tickers = list(FAN_TICKERS.keys())
metrics_df = fetch_underlying_metrics(
    tickers, history_period="1y", rate_limit_sleep=RATE_LIMIT_SLEEP
)
assert not metrics_df.empty, "Could not fetch underlying data."

overview_rows = []
pool_parts: list[pd.DataFrame] = []
for _, m in metrics_df.iterrows():
    tkr = m["ticker"]
    spot = float(m["spot"])
    target_mean = float(m["target_mean"]) if pd.notna(m["target_mean"]) else None
    target_high = float(m["target_high"]) if pd.notna(m["target_high"]) else None
    lo, hi, src = _derive_zone(spot, target_mean, target_high, FAN_TICKERS.get(tkr))

    chain = fetch_fan_candidates(
        tkr,
        spot=spot,
        strike_range=(lo, hi),
        min_dte=MIN_DTE,
        max_dte=MAX_DTE,
        min_open_interest=max(25, int(MIN_OI * 0.5)),
        min_volume=max(10, int(MIN_VOLUME * 0.5)),
        max_spread_pct=MAX_SPREAD * 1.6,
        rate_limit_sleep=RATE_LIMIT_SLEEP,
    )

    strict = score_short_upside_candidates(chain, m, relaxed=False)
    scored = strict
    used_relaxed = False
    if strict.empty:
        relaxed = score_short_upside_candidates(chain, m, relaxed=True)
        if not relaxed.empty:
            scored = relaxed
            used_relaxed = True

    if not scored.empty:
        top_per_exp = (
            scored.sort_values(["upside_score", "upside_multiple"], ascending=False)
            .groupby("expiration", as_index=False)
            .head(1)
            .reset_index(drop=True)
        )
        pool_parts.append(top_per_exp)

    overview_rows.append(
        {
            "ticker": tkr,
            "spot": spot,
            "target": target_mean,
            "ret_3m": m["ret_3m"],
            "hv_30": m["hv_30"],
            "rsi_14": m["rsi_14"],
            "next_earnings": m["next_earnings_date"],
            "zone": f"${lo:,.0f}-${hi:,.0f}",
            "zone_source": src,
            "contracts_fetched": len(chain),
            "contracts_scored": len(scored),
            "mode": "relaxed" if used_relaxed else "strict",
        }
    )

overview_df = pd.DataFrame(overview_rows).sort_values(
    "contracts_scored", ascending=False
)
display(Markdown("### Universe Scan Summary"))
display_table(
    overview_df.rename(
        columns={
            "ticker": "Ticker",
            "spot": "Spot",
            "target": "Target",
            "ret_3m": "3m Ret",
            "hv_30": "HV",
            "rsi_14": "RSI",
            "next_earnings": "Next Earnings",
            "zone": "Strike Zone",
            "zone_source": "Zone Source",
            "contracts_fetched": "Fetched",
            "contracts_scored": "Passed",
            "mode": "Mode",
        }
    ),
    caption="Ticker-level intake and filter pass-through",
    format_dict={
        "Spot": "${:,.2f}",
        "Target": "${:,.0f}",
        "3m Ret": "{:+.1%}",
        "HV": "{:.0%}",
        "RSI": "{:.0f}",
    },
)

if pool_parts:
    pool = pd.concat(pool_parts, ignore_index=True)
    pool = pool.sort_values(
        ["upside_score", "upside_multiple", "liq_quality"], ascending=False
    ).reset_index(drop=True)
else:
    pool = pd.DataFrame()

assert not pool.empty, (
    "No candidates survived filters. Loosen thresholds or add tickers."
)
display(
    Markdown(
        f"**Candidate pool:** {len(pool)} contracts across {pool['ticker'].nunique()} tickers"
    )
)
display_table(
    pool.head(20)[
        [
            "ticker",
            "contract",
            "dte",
            "strike",
            "entry_fill",
            "cost_1ct_exec",
            "delta",
            "pop",
            "iv_hv",
            "upside_multiple",
            "upside_score",
            "flags",
        ]
    ].rename(
        columns={
            "ticker": "Ticker",
            "contract": "Contract",
            "dte": "DTE",
            "strike": "Strike",
            "entry_fill": "Entry",
            "cost_1ct_exec": "Cost 1ct",
            "delta": "Delta",
            "pop": "PoP",
            "iv_hv": "IV/HV",
            "upside_multiple": "Upside x",
            "upside_score": "Upside Score",
            "flags": "Flags",
        }
    ),
    caption="Top candidates before portfolio optimization",
    format_dict={
        "Strike": "${:,.0f}",
        "Entry": "${:,.2f}",
        "Cost 1ct": "${:,.0f}",
        "Delta": "{:.2f}",
        "PoP": "{:.0%}",
        "IV/HV": "{:.2f}x",
        "Upside x": "{:.2f}x",
        "Upside Score": "{:.1f}",
    },
)


if ENABLE_CHARTS:
    display(Markdown("### Candidate Diagnostics"))

    candidate_chart_specs = [
        ("01_candidate_score_rank", plot_candidate_score_rank, (pool, TOP_N_CANDIDATES)),
        ("02_pop_vs_upside", plot_pop_vs_upside, (pool, TOP_N_CANDIDATES)),
        ("03_iv_hv_vs_score", plot_iv_hv_vs_score, (pool, TOP_N_CANDIDATES)),
    ]

    if CHART_MODE == "full":
        candidate_chart_specs.extend(
            [
                ("04_dte_distribution", plot_dte_distribution, (pool,)),
                ("05_ticker_pass_heatmap", plot_ticker_pass_heatmap, (overview_df,)),
                ("06_liquidity_spread", plot_liquidity_spread, (pool, TOP_N_CANDIDATES)),
                ("07_flag_frequency", plot_flag_frequency, (pool,)),
            ]
        )

    for chart_name, chart_fn, chart_args in candidate_chart_specs:
        chart = chart_fn(*chart_args)
        show_and_optionally_save(chart, chart_name, RUN_STAMP)


### Universe Scan Summary

,Ticker,Spot,Target,3m Ret,HV,RSI,Next Earnings,Strike Zone,Zone Source,Fetched,Passed,Mode
1,MSFT,$403.85,$596,-20.0%,42%,35,2026-01-28,$390-$600,analyst,198,27,strict
0,AAPL,$277.14,$293,+3.0%,23%,83,2026-01-29,$270-$350,analyst,106,26,strict
5,GOOGL,$311.02,$372,+7.3%,21%,32,2026-02-04,$300-$440,analyst,142,20,strict
6,META,$669.75,$860,+6.1%,42%,55,2026-01-28,$650-$860,analyst,114,19,strict
3,TSLA,$426.51,$418,-4.2%,38%,40,2026-01-28,$410-$420,analyst,15,14,relaxed
4,AMZN,$204.36,$283,-17.7%,33%,25,2026-02-05,$200-$280,analyst,128,12,strict
7,NFLX,$79.89,$111,-28.7%,23%,38,2026-01-20,$80-$110,analyst,122,12,strict
11,MU,$410.75,$382,+62.2%,77%,53,2025-12-17,$400-$420,analyst,22,9,relaxed
14,GEV,$823.01,$826,+42.1%,45%,79,2026-01-28,$800-$830,analyst,8,7,relaxed
15,KLAC,"$1,492.70","$1,665",+22.8%,74%,50,2026-01-29,"$1,450-$1,950",analyst,8,4,relaxed


**Candidate pool:** 62 contracts across 18 tickers

,Ticker,Contract,DTE,Strike,Entry,Cost 1ct,Delta,PoP,IV/HV,Upside x,Upside Score,Flags
0,CMI,CMI260918C00690000,219,$690,$30.08,"$3,008",0.37,23%,0.71x,9.64x,634.7,--
1,KLAC,KLAC260515C01700000,93,"$1,700",$81.97,"$8,197",0.37,22%,0.68x,8.82x,579.8,--
2,MU,MU260515C00420000,93,$420,$56.03,"$5,603",0.56,29%,0.94x,6.80x,452.2,--
3,KLAC,KLAC260618C01600000,127,"$1,600",$136.59,"$13,659",0.49,27%,0.69x,6.46x,429.0,--
4,CMI,CMI260618C00600000,127,$600,$47.09,"$4,710",0.57,34%,0.71x,6.39x,425.0,--
5,MU,MU260618C00420000,127,$420,$66.30,"$6,630",0.58,28%,0.94x,6.08x,402.5,--
6,TSM,TSM260515C00400000,93,$400,$25.00,"$2,500",0.45,27%,1.26x,5.34x,358.3,--
7,MU,MU260717C00410000,156,$410,$78.58,"$7,858",0.61,29%,0.95x,5.41x,356.6,--
8,GEV,GEV260515C00820000,93,$820,$90.59,"$9,058",0.57,32%,1.18x,5.30x,351.7,rsi_hot
9,WDC,WDC260618C00270000,127,$270,$58.51,"$5,851",0.63,29%,0.85x,5.28x,347.6,--


In [12]:
# Portfolio optimizer (budget + concentration + correlation constraints)
def _is_corr_related(t1: str, t2: str, corr_mx: pd.DataFrame, threshold: float) -> bool:
    if corr_mx.empty:
        return False
    if t1 not in corr_mx.index or t2 not in corr_mx.columns:
        return False
    v = corr_mx.loc[t1, t2]
    if pd.isna(v):
        return False
    return abs(float(v)) >= threshold


def optimize_buy_list(
    pool_df: pd.DataFrame, budget: float, corr_mx: pd.DataFrame
) -> tuple[pd.DataFrame, float]:
    if pool_df.empty:
        return pd.DataFrame(), budget

    candidates = pool_df.sort_values(
        ["upside_score", "upside_multiple", "liq_quality"], ascending=False
    ).reset_index(drop=True)
    selected = []
    budget_left = float(budget)
    spent_by_ticker: dict[str, float] = {}

    for _, row in candidates.iterrows():
        cost = float(row["cost_1ct_exec"])
        if not np.isfinite(cost) or cost <= 0:
            continue

        tkr = row["ticker"]
        pos_cap = budget * POSITION_CAP_PCT
        tkr_cap = budget * TICKER_CAP_PCT

        qty_by_pos = int(pos_cap // cost)
        qty_by_tkr = int(max(tkr_cap - spent_by_ticker.get(tkr, 0.0), 0.0) // cost)
        qty_by_budget = int(budget_left // cost)
        qty = min(qty_by_pos, qty_by_tkr, qty_by_budget, MAX_CONTRACTS_PER_LINE)
        if qty <= 0:
            continue

        # Enforce correlation cluster exposure cap by reducing qty when necessary.
        while qty > 0:
            line_cost = qty * cost
            deployed_now = budget - budget_left
            prospective_total = deployed_now + line_cost

            correlated_peer_cost = 0.0
            for peer_tkr, peer_spent in spent_by_ticker.items():
                if _is_corr_related(tkr, peer_tkr, corr_mx, CORR_THRESHOLD):
                    correlated_peer_cost += peer_spent

            if prospective_total <= 0:
                break

            # Correlation cap only applies when there is correlated peer exposure.
            if correlated_peer_cost <= 0:
                break

            corr_cluster_ratio = (correlated_peer_cost + line_cost) / prospective_total
            if corr_cluster_ratio <= MAX_CORRELATED_EXPOSURE_PCT:
                break
            qty -= 1

        if qty <= 0:
            continue

        line = row.copy()
        line["qty"] = int(qty)
        line["line_cost"] = float(qty * cost)
        line["theta_dollars_day"] = float(row["theta"]) * 100.0 * qty
        line["max_loss"] = float(qty * cost)
        selected.append(line)

        spent_by_ticker[tkr] = spent_by_ticker.get(tkr, 0.0) + float(qty * cost)
        budget_left -= float(qty * cost)

        if len(selected) >= MAX_LINES:
            break

    if not selected:
        return pd.DataFrame(), float(budget)

    buy_df = pd.DataFrame(selected).reset_index(drop=True)
    buy_df.insert(0, "#", range(1, len(buy_df) + 1))
    total_deployed = float(buy_df["line_cost"].sum())
    buy_df["portfolio_weight"] = (
        buy_df["line_cost"] / total_deployed if total_deployed > 0 else 0.0
    )
    buy_df["stop_price"] = buy_df["entry_fill"] * 0.50
    buy_df["tp_50"] = buy_df["entry_fill"] * 1.50
    buy_df["tp_100"] = buy_df["entry_fill"] * 2.00
    return buy_df, float(budget_left)


corr_mx = build_correlation_matrix(
    pool["ticker"].unique().tolist(), period="6mo", rate_limit_sleep=RATE_LIMIT_SLEEP
)
buy_df, budget_left = optimize_buy_list(pool, float(MY_BUDGET), corr_mx)

if buy_df.empty:
    display(Markdown("## No positions passed final portfolio constraints."))
else:
    total_deployed = float(buy_df["line_cost"].sum())
    utilization = total_deployed / float(MY_BUDGET)
    total_theta = float(buy_df["theta_dollars_day"].sum())
    avg_pop = float(buy_df["pop"].mean())
    avg_upside = float(buy_df["upside_multiple"].mean())

    display(Markdown("---"))
    display(Markdown("# BUY LIST (v2)"))
    display(Markdown(f"*Generated {datetime.now().strftime('%b %d, %Y %H:%M')}*"))

    display(
        Markdown(
            f"| Metric | Value |\n|---|---|\n"
            f"| Budget | ${MY_BUDGET:,.0f} |\n"
            f"| Deployed | ${total_deployed:,.0f} ({utilization:.0%}) |\n"
            f"| Cash reserve | ${budget_left:,.0f} |\n"
            f"| Positions | {len(buy_df)} lines, {buy_df['ticker'].nunique()} tickers |\n"
            f"| Avg PoP | {avg_pop:.0%} |\n"
            f"| Avg upside (scenario) | {avg_upside:.2f}x |\n"
            f"| Daily theta burn | ${total_theta:+,.0f} |\n"
            f"| Max loss (premium) | ${total_deployed:,.0f} |"
        )
    )

    out_cols = [
        "#",
        "ticker",
        "contract",
        "qty",
        "expiration",
        "dte",
        "strike",
        "entry_fill",
        "cost_1ct_exec",
        "line_cost",
        "delta",
        "pop",
        "upside_multiple",
        "upside_score",
        "iv_hv",
        "be_pct",
        "flags",
        "stop_price",
        "tp_50",
        "tp_100",
        "portfolio_weight",
    ]
    display_table(
        buy_df[out_cols].rename(
            columns={
                "ticker": "Ticker",
                "contract": "Contract",
                "qty": "Qty",
                "expiration": "Exp",
                "dte": "DTE",
                "strike": "Strike",
                "entry_fill": "Entry",
                "cost_1ct_exec": "Cost /ct",
                "line_cost": "Line Cost",
                "delta": "Delta",
                "pop": "PoP",
                "upside_multiple": "Upside x",
                "upside_score": "Score",
                "iv_hv": "IV/HV",
                "be_pct": "BE Dist",
                "flags": "Flags",
                "stop_price": "Stop -50%",
                "tp_50": "TP +50%",
                "tp_100": "TP +100%",
                "portfolio_weight": "Weight",
            }
        ),
        caption="Buy to Open list with execution-adjusted pricing",
        format_dict={
            "Strike": "${:,.0f}",
            "Entry": "${:,.2f}",
            "Cost /ct": "${:,.0f}",
            "Line Cost": "${:,.0f}",
            "Delta": "{:.2f}",
            "PoP": "{:.0%}",
            "Upside x": "{:.2f}x",
            "Score": "{:.1f}",
            "IV/HV": "{:.2f}x",
            "BE Dist": "{:+.1%}",
            "Stop -50%": "${:,.2f}",
            "TP +50%": "${:,.2f}",
            "TP +100%": "${:,.2f}",
            "Weight": "{:.0%}",
        },
    )

    if ENABLE_CHARTS:
        display(Markdown("### Portfolio Diagnostics"))

        portfolio_chart_specs = [
            ("08_portfolio_weight", plot_portfolio_weight, (buy_df,)),
            ("09_theta_contribution", plot_theta_contribution, (buy_df,)),
            ("10_expiration_ladder", plot_expiration_ladder, (buy_df,)),
        ]

        if CHART_MODE == "full":
            portfolio_chart_specs.extend(
                [
                    (
                        "11_correlation_matrix",
                        plot_correlation_matrix,
                        (corr_mx, buy_df["ticker"].dropna().astype(str).unique().tolist()),
                    ),
                    ("12_entry_stop_tp", plot_entry_stop_tp, (buy_df,)),
                ]
            )

        for chart_name, chart_fn, chart_args in portfolio_chart_specs:
            chart = chart_fn(*chart_args)
            show_and_optionally_save(chart, chart_name, RUN_STAMP)

    display(
        Markdown(
            "## Exit Rules\n"
            "| Trigger | Action |\n"
            "|---|---|\n"
            "| Option down 50% | Close. Do not average down. |\n"
            "| Option up 50% | Take partial (about one-third), move stop to entry. |\n"
            "| Option up 100% | Sell half. Let remainder run with trailing stop. |\n"
            "| DTE < 5 days | Close or roll. Avoid last-week theta cliff. |\n"
            "| Daily portfolio loss > 6% of budget | Pause new entries for the day. |"
        )
    )


---

# BUY LIST (v2)

*Generated Feb 11, 2026 15:08*

| Metric | Value |
|---|---|
| Budget | $15,000 |
| Deployed | $14,627 (98%) |
| Cash reserve | $373 |
| Positions | 2 lines, 2 tickers |
| Avg PoP | 26% |
| Avg upside (scenario) | 8.22x |
| Daily theta burn | $-79 |
| Max loss (premium) | $14,627 |

,#,Ticker,Contract,Qty,Exp,DTE,Strike,Entry,Cost /ct,Line Cost,Delta,PoP,Upside x,Score,IV/HV,BE Dist,Flags,Stop -50%,TP +50%,TP +100%,Weight
0,1,CMI,CMI260918C00690000,3,2026-09-18,219,$690,$30.08,"$3,008","$9,024",0.37,23%,9.64x,634.7,0.71x,+19.8%,--,$15.04,$45.12,$60.16,62%
1,2,MU,MU260515C00420000,1,2026-05-15,93,$420,$56.03,"$5,603","$5,603",0.56,29%,6.80x,452.2,0.94x,+15.9%,--,$28.02,$84.05,$112.06,38%


## Exit Rules
| Trigger | Action |
|---|---|
| Option down 50% | Close. Do not average down. |
| Option up 50% | Take partial (about one-third), move stop to entry. |
| Option up 100% | Sell half. Let remainder run with trailing stop. |
| DTE < 5 days | Close or roll. Avoid last-week theta cliff. |
| Daily portfolio loss > 6% of budget | Pause new entries for the day. |